#TopEditor - traitements NER et enrichissements des textes TEI

Le dossier du projet se trouve ici :
```
/content/drive/MyDrive/Colab Notebooks/TopEditor/
```
Description des sous-dossiers associés aux traitements:
* **NERmodels** : les modèles NER produits de reperage d'entités nommées (et autres)

* **data/TEI/Final/input**: fichiers TEI stylé via Word customisé Metopes et converti en schéma métopes (via XMLMind)

* **data/TEI/Final/clean**: fichiers TEI du dossier input legerement "nettoyés" manuellement du bruit produit par Word après consignes Métopes

* **data/TEI/Final/tagged**: fichiers TEI avec balisage automatisé via un modèle NER custom

* **data/TEI/Final/reference_pers**: fichiers tableurs avec les identifiants personnes à désambiguiser manuellement

* **data/TEI/Final/reference_place**: fichiers tableurs avec les identifiants place (sens restreint) à désambiguiser manuellement

* **data/TEI/Final/tagged/updated_person**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes

* **data/TEI/Final/tagged/updated_person_upated_place**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes et pour les lieux

* **data/TEI/Final/tagged_corrected_Sara**: fichiers TEI dans updated_person_upated_place avec la correction manuelle par Sara via Oxygène des balises fautives

* **data/TEI/Final/tagged_corrected_Sara/TagAugmented** fichiers dans tagged_corrected_Sara avec un balisage automatisé par reglès des dates déjà reperées par le modèle NER et pas du tout repérées




## Lieux - collaciones, calles, CTMB (tagged -> reference_place, updated_person_updated_place)

Gestion du référentiel des lieux cités

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Création d'un tableur de lieux à partir des noms de collaciones, rues, POI repérés dans les textes (TEI) et réinjection des identifiants dans le XML

In [3]:
import os
from lxml import etree
import pandas as pd
import re

def get_full_text(element):
    """Récupère tout le texte (y compris sous-éléments) dans une balise."""
    parts = [element.text] if element.text else []
    for sub_element in element:
        parts.append(get_full_text(sub_element))
        if sub_element.tail:
            parts.append(sub_element.tail)
    return ''.join(parts).strip()

def create_and_inject_place_id(paragraph, namespace):
    """Crée et injecte des identifiants uniques pour les noms de lieux."""
    for place_type in ['street_name', 'area_name', 'poi_name']:
      place_elements = paragraph.findall(f'.//tei:rs[@type="{place_type}"]', namespaces=namespace)
      for element in place_elements:
          place_text = get_full_text(element)
          place_id = place_text.replace(' ', '_').lower()
          #print(place_id)
          element.set('ref', "#"+place_id)

def extract_place_data(paragraph, namespace):
    """Extrait les noms de lieux d'un paragraphe donné."""
    places = []
    for place_type in ['street_name', 'area_name', 'poi_name']:
        elements = paragraph.findall(f'.//tei:rs[@type="{place_type}"]', namespaces=namespace)
        for element in elements:
            place_text = get_full_text(element)
            placeID = f"{place_text}".replace(" ", "_").lower()
            places.append({
                'Nombre': place_text,
                'Tipo': place_type,
                'identificador': placeID,
                'identificador_desambiguar': placeID,
                'ID parrafo': paragraph.get('{http://www.w3.org/XML/1998/namespace}id'),
                'Parrafo': get_full_text(paragraph)
            })
    return places

def process_file_places(file_path, namespace):
    """Traite un fichier XML pour extraire les noms de lieux."""
    tree = etree.parse(file_path)
    root = tree.getroot()
    body = root.find('.//tei:body', namespaces=namespace)
    if body is None:
        print(f"Aucun body trouvé dans le fichier {file_path}")
        return []

    data = []
    paragraphs = body.findall('.//tei:p', namespaces=namespace)
    for paragraph in paragraphs:
        data.extend(extract_place_data(paragraph, namespace))
        create_and_inject_place_id(paragraph, namespace)

    # Sauvegarder le fichier XML mis à jour
    updated_file_path = file_path.replace('.xml', '_updated_place.xml')
    tree.write(updated_file_path, encoding='utf-8', xml_declaration=True)

    return data

def save_to_excel(data, output_file):
    """Enregistre les données dans un fichier Excel."""
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False, sheet_name='Occurrences')

def extract_place_context(xml_folder, output_folder):
    """Parcourt les fichiers XML dans un dossier pour extraire et injecter les noms de lieux."""
    namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

    for file_name in os.listdir(xml_folder):
        if file_name.endswith('_clean_updated_person.xml'):
            file_path = os.path.join(xml_folder, file_name)
            print(f"Traitement du fichier : {file_path}")

            try:
                data = process_file_places(file_path, namespace)

                if data:
                    output_file = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}_places_context.xlsx")
                    save_to_excel(data, output_file)
                    print(f"Extraction terminée pour {file_name}. Résultats sauvegardés dans {output_file}")
                else:
                    print(f"Aucune donnée extraite pour {file_name}.")

            except etree.XMLSyntaxError as e:
                print(f"Erreur lors de l'analyse du fichier {file_name}: {e}")


# Paramètres du script
xml_folder = '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged'
out_tableur = '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/reference_place'

extract_place_context(xml_folder, out_tableur)


Traitement du fichier : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean_updated_person.xml
Extraction terminée pour semtags_2combiné_T2 clj. San Pedro_clean_updated_person.xml. Résultats sauvegardés dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/reference_place/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_places_context.xlsx


Ver notebook de "control"

##Après correction manuelle:

### TODO supprimer les faux positifs (place) des tableurs de Jean des fichiers XML via l'identifiant (attribut ref)

### TODO Génération de l'index TEI pour les lieux à partir des fichiers TEI et les tableurs corrigés